In [ ]:
# 去除阴影尝试
import numpy as np
from matplotlib import pyplot as plt
import cv2

image = cv2.imread("../test_images/1.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 光滑
gray = cv2.GaussianBlur(gray, (7, 7), 0)

 # 闭运算
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
close = cv2.morphologyEx(gray, cv2.MORPH_CLOSE, kernel, iterations=9)

# 闭运算结果减去原灰度图像
subtracted = cv2.subtract(close, gray)
# 取反
inverted = cv2.bitwise_not(subtracted)
# 归一化
normalized = cv2.normalize(inverted, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)

edge = cv2.Canny(normalized, 50, 100)  # Canny方法边缘检测

contour = image.copy()
(cnts, _) = cv2.findContours(edge, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  # 轮廓检测
cv2.drawContours(contour, cnts, -1, (0, 255, 0), 2)  # 绘制轮廓

plt.figure(2)
plt.imshow(inverted, cmap='gray')
plt.figure(3)
plt.imshow(edge, cmap='gray')
plt.figure(4)
plt.imshow(cv2.cvtColor(contour, cv2.COLOR_BGR2RGB))

In [ ]:
# 二值化尝试
import numpy as np
from matplotlib import pyplot as plt
import cv2

# 二值化
# ret, binary = cv2.threshold(normalized, 150, 255, cv2.THRESH_BINARY)

# ret, binary = cv2.threshold(normalized, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
binary = cv2.adaptiveThreshold(normalized, 255, 
                                        cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                        cv2.THRESH_BINARY_INV, 91, 6)

# 闭运算
kernel1 = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
kernel2 = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
erode = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel1, iterations=3)
dilate = cv2.morphologyEx(erode, cv2.MORPH_CLOSE, kernel2, iterations=5)

plt.figure(1)
plt.imshow(binary, cmap='gray')
plt.figure(2)
plt.imshow(erode, cmap='gray')
plt.figure(3)
plt.imshow(dilate, cmap='gray')

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import cv2

count = 0  # 积木个数
margin = 10  # 裁剪边距
draw_rect = image.copy()
for i, contour in enumerate(cnts):
    area = cv2.contourArea(contour)  # 计算包围形状的面积
    if area < 25:  # 过滤面积小于某个值的形状
        continue
    # print(area)
    rect = cv2.minAreaRect(contour)  # 检测轮廓最小外接矩形，得到最小外接矩形的中心(x,y), (宽,高), 旋转角度
    box = np.intp(cv2.boxPoints(rect))  # 获取最小外接矩形的4个顶点坐标
    cv2.drawContours(draw_rect, [box], 0, (255, 0, 0), 2)  # 绘制轮廓最小外接矩形

    h, w = image.shape[:2]  # 原图像的高和宽
    rect_w, rect_h = int(rect[1][0]) + 1, int(rect[1][1]) + 1  # 最小外接矩形的宽和高
    if rect_w <= 100 or rect_h <= 150:
        continue
    count += 1
    if rect_w <= rect_h:
        x, y = int(box[1][0]), int(box[1][1])  # 旋转中心
        M2 = cv2.getRotationMatrix2D((x, y), rect[2], 1)
        rotated_image = cv2.warpAffine(image, M2, (w * 2, h * 2))
        y1, y2 = y - margin if y - margin > 0 else 0, y + rect_h + margin + 1
        x1, x2 = x - margin if x - margin > 0 else 0, x + rect_w + margin + 1
        rotated_canvas = rotated_image[y1: y2, x1: x2]
    else:
        x, y = int(box[2][0]), int(box[2][1])  # 旋转中心
        M2 = cv2.getRotationMatrix2D((x, y), rect[2] + 90, 1)
        rotated_image = cv2.warpAffine(image, M2, (w * 2, h * 2))
        y1, y2 = y - margin if y - margin > 0 else 0, y + rect_w + margin + 1
        x1, x2 = x - margin if x - margin > 0 else 0, x + rect_h + margin + 1
        rotated_canvas = rotated_image[y1: y2, x1: x2]

plt.figure(1)
plt.imshow(cv2.cvtColor(draw_rect, cv2.COLOR_BGR2RGB))